In [1]:
import sys
import re
import traceback
import IPython

# replace the trackback handler to hide paths on server
def exception_hook(*args, **kwargs):
    lines = traceback.format_exception(*sys.exc_info())
    lines = [lines[0]] + lines[2:]
    lines = [re.sub(r'^.+(/cmdy/.+\.py)', r'/path/.../to\1', line) for line in lines]
    lines = [re.sub(r'^.+(/miniconda3/.+\.py)', r'/path/.../to\1', line) for line in lines]
    sys.stderr.write(''.join(lines))
    
IPython.core.interactiveshell.InteractiveShell.showtraceback = exception_hook

# Usage

## Basic usage

In [2]:
from cmdy import ls

In [3]:
print(ls())

LICENSE
README.md
README.rst
cmdy
demo.ipynb
pyproject.toml
pytest.ini
requirements.txt
setup.py
test.py
tests



In [4]:
for line in ls().iter():
    print('Got:', line, end='')

Got: LICENSE
Got: README.md
Got: README.rst
Got: cmdy
Got: demo.ipynb
Got: pyproject.toml
Got: pytest.ini
Got: requirements.txt
Got: setup.py
Got: test.py
Got: tests


## With non-keyword arguments

In [5]:
from cmdy import tar
print(tar("cvf", "/tmp/test.tar", "./cmdy"))

./cmdy/
./cmdy/__init__.py
./cmdy/cmdy_plugin.py
./cmdy/cmdy_util.py
./cmdy/cmdy_result.py
./cmdy/__pycache__/
./cmdy/__pycache__/cmdy_result.cpython-37.pyc
./cmdy/__pycache__/__init__.cpython-37.pyc
./cmdy/__pycache__/cmdy_plugin.cpython-37.pyc
./cmdy/__pycache__/cmdy_util.cpython-37.pyc



## With keyword arguments

In [6]:
from cmdy import curl
curl("http://duckduckgo.com/", o="/tmp/page.html", silent=True)
# curl http://duckduckgo.com/ -o /tmp/page.html --silent

<CmdyResult: ['curl', 'http://duckduckgo.com/', '-o', '/tmp/page.html', '--silent']>

### Order keyword arguments

In [7]:
curl("http://duckduckgo.com/", "-o", "/tmp/page.html", "--silent")

<CmdyResult: ['curl', 'http://duckduckgo.com/', '-o', '/tmp/page.html', '--silent']>

In [8]:
# or
from diot import OrderedDiot
kwargs = OrderedDiot()
kwargs.silent = True
kwargs.o = '/tmp/page.html'
curl("http://duckduckgo.com/", kwargs)
# You can also use collections.OrderedDict

<CmdyResult: ['curl', 'http://duckduckgo.com/', '--silent', '-o', '/tmp/page.html']>

### Prefix and separator for keyword arguments

### Different combinations of prefices and separators in one command

### Duplicated keys for list arguments:

In [9]:
from cmdy import sort
print(sort(k=['1,1', '2,2'], t='_', _='./.editorconfig', cmdy_dupkey=True))
# sort -k 1,1 -k 2,2 ./.editorconfig


[*]
end_of_line          = lf
indent_size          = 4
indent_style         = tab
insert_final_newline = true
root = true
tab_width            = 4



## Return code and exception

In [10]:
from cmdy import x
x()

Traceback (most recent call last):
  File "<ipython-input-10-092cc5b72e61>", line 2, in <module>
    x()
/path/.../to/cmdy/__init__.py", line 71, in __call__
    ready_cfgargs, ready_popenargs, will)
/path/.../to/cmdy/cmdy_result.py", line 48, in __new__
    result = holding.run()
/path/.../to/cmdy/cmdy_plugin.py", line 514, in run
    return orig_run(self, wait)
/path/.../to/cmdy/cmdy_plugin.py", line 400, in run
    return orig_run(self, wait)
/path/.../to/cmdy/cmdy_result.py", line 146, in run
    ret = CmdyResult(self._run(), self)
/path/.../to/cmdy/cmdy_result.py", line 114, in _run
    raise CmdyExecNotFoundError(str(fnfe)) from None
cmdy.cmdy_util.CmdyExecNotFoundError: [Errno 2] No such file or directory: 'x': 'x'


In [11]:
from cmdy import ls
ls('non-existing-file')

Traceback (most recent call last):
  File "<ipython-input-11-132683fc2227>", line 2, in <module>
    ls('non-existing-file')
/path/.../to/cmdy/__init__.py", line 71, in __call__
    ready_cfgargs, ready_popenargs, will)
/path/.../to/cmdy/cmdy_result.py", line 50, in __new__
    return result.wait()
/path/.../to/cmdy/cmdy_result.py", line 208, in wait
    raise CmdyReturnCodeError(self)
cmdy.cmdy_util.CmdyReturnCodeError: Unexpected RETURN CODE 2, expecting: [0]

  [   PID] 197541

  [   CMD] ['ls', 'non-existing-file']

  [STDOUT] 

  [STDERR] ls: cannot access non-existing-file: No such file or directory



### Don't raise exception but store the return code

In [13]:
from cmdy import ls
result = ls('non-existing-file', cmdy_raise=False)
result.rc

2

### Tolerance on return code

In [14]:
from cmdy import ls
ls('non-existing-file', cmdy_okcode='0,2').rc # or [0,2]

2

### Timeouts

In [15]:
from cmdy import sleep
sleep(3, cmdy_timeout=1)

Traceback (most recent call last):
  File "<ipython-input-15-6f9922c1c236>", line 2, in <module>
    sleep(3, cmdy_timeout=1)
/path/.../to/cmdy/__init__.py", line 71, in __call__
    ready_cfgargs, ready_popenargs, will)
/path/.../to/cmdy/cmdy_result.py", line 50, in __new__
    return result.wait()
/path/.../to/cmdy/cmdy_result.py", line 205, in wait
    ) from None
cmdy.cmdy_util.CmdyTimeoutError: Timeout after 1 seconds.


## Redirections

In [16]:
from cmdy import cat
cat('./pytest.ini').redirect() > '/tmp/pytest.ini'
print(cat('/tmp/pytest.ini'))

[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1



### Appending

In [17]:
# r short for redirect
cat('./pytest.ini').r() >> '/tmp/pytest.ini'
print(cat('/tmp/pytest.ini'))

[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1
[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1



### Redirecting to a file handler

In [18]:
from cmdy import cat
f = open('/tmp/pytest.ini', 'w')
# executing fails to detect future action in with block with ipython
# but feel free to write with block in regular python
cat('./pytest.ini').r() > f
f.close()
print(cat('/tmp/pytest.ini'))

[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1



### STDIN, STDOUT and/or STDERR redirections

In [19]:
from cmdy import STDIN, STDOUT, STDERR, DEVNULL

c = cat().r(STDIN) < '/tmp/pytest.ini'
print(c)

[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1



In [20]:
# Mixed
c = cat().r(STDIN, STDOUT) ^ '/tmp/pytest.ini' > DEVNULL
# we can't fetch result from a redirected pipe
print(c.stdout)

# Why not '<' for STDIN?
# Because the priority of the operator is not in sequential order.
# We can use < for STDIN, but we need to ensure it runs first
c = (cat().r(STDIN, STDOUT) < '/tmp/pytest.ini') > DEVNULL
print(c.stdout)

# A simple rule for multiple redirections to always use ">" in the last place

None
None


In [21]:
# Redirect stderr to stdout
from cmdy import bash
c = bash(c="cat 1>&2").r(STDIN, STDERR) ^ '/tmp/pytest.ini' > STDOUT
print(c.stdout)

[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1



In [22]:
# All at the same time
c = bash(c="cat 1>&2").r(STDIN, STDOUT, STDERR) ^ '/tmp/pytest.ini' ^ DEVNULL > STDOUT
print(c.stdout)
print(c.stderr)

None
None


## Pipings

In [23]:
from cmdy import ls, grep
c = ls().pipe() | grep('README')
print(c)

README.md
README.rst



In [24]:
# p short for pipe
c = ls().p() | grep('README').p() | grep('md')
print(c)
print(c.piped_strcmds)

README.md

['ls', 'grep README', 'grep md']


In [25]:
from cmdy.cmdy_result import EVENT
# !!! Pipings should be consumed immediately!
# !!! DO NOT do this
ls().p()
ls() # <- Will not run as expected
# All commands will be locked as holding until pipings are consumed
EVENT.clear()
print(ls())

# See Advanced/Holdings if you want to hold a piping command for a while

LICENSE
README.md
README.rst
cmdy
demo.ipynb
pyproject.toml
pytest.ini
requirements.txt
setup.py
test.py
tests



## Running command in foreground

In [26]:
ls().fg()

LICENSE
README.md
README.rst
cmdy
demo.ipynb
pyproject.toml
pytest.ini
requirements.txt
setup.py
test.py
tests


<CmdyResult: ['ls']>

In [27]:
from cmdy import tail
tail('/tmp/pytest.ini', f=True).fg()
# This mimics the `tail -f` program
# You will see the content comes out one after another
# and then program hangs

[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1


Traceback (most recent call last):
  File "<ipython-input-27-a1b136f3783e>", line 2, in <module>
    tail('/tmp/pytest.ini', f=True).fg()
/path/.../to/cmdy/cmdy_plugin.py", line 169, in wrapper
    return func(self, *args, **kwargs)
/path/.../to/cmdy/cmdy_plugin.py", line 391, in foreground
    return self.run()
/path/.../to/cmdy/cmdy_plugin.py", line 514, in run
    return orig_run(self, wait)
/path/.../to/cmdy/cmdy_plugin.py", line 404, in run
    curio.run(Fg._feed(ret, self.data.foreground.poll_interval))
/path/.../to/miniconda3/lib/python3.7/site-packages/curio/kernel.py", line 826, in run
    return kernel.run(corofunc, *args)
/path/.../to/miniconda3/lib/python3.7/site-packages/curio/kernel.py", line 147, in run
    task = self._runner(coro)
/path/.../to/miniconda3/lib/python3.7/site-packages/curio/kernel.py", line 645, in kernel_run
    events = selector_select(timeout)
/path/.../to/miniconda3/lib/python3.7/selectors.py", line 468, in select
    fd_event_list = self._selector.po

In [ ]:
# You also write an `echo-like` program easily
# 
# This will not run here
# !!! NOT RUN
# Save it to a file and run with python interpreter

# from cmdy import cat
# cat().fg(stdin=True)

## Iterating on output

In [28]:
for line in ls().iter():
    print(line, end='')

LICENSE
README.md
README.rst
cmdy
demo.ipynb
pyproject.toml
pytest.ini
requirements.txt
setup.py
test.py
tests


### Iterating on stderr

In [29]:
from cmdy import bash, STDERR
for line in bash(c="cat /tmp/pytest.ini 1>&2").iter(STDERR):
    print(line, end='')

[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1


### Getting live output

In [34]:
# Like we did for `tail -f` program
# This time, we can do something with each output line

# Let's use a thread to write content to a file
# And we try to get the live contents using cmdy
import time
from threading import Thread
def live_write(file, n):
    
    with open(file, 'w', buffering=1) as f:
        # Let's write something every half second
        for i in range(n):
            f.write(str(i) + '\n')
            time.sleep(.5)
            
test_file = '/tmp/tail-f.txt'
Thread(target=live_write, args=(test_file, 10)).start()

from cmdy import tail

tail_iter = tail(f=True, _=test_file).iter()

for line in tail_iter:
    # Do whatever you want with the line
    print('We got:', line, end='')
    if line.strip() == '8':
        break
        
# make sure thread ends
time.sleep(2)

We got: 0
We got: 1
We got: 2
We got: 3
We got: 4
We got: 5
We got: 6
We got: 7
We got: 8


In [35]:
# What about timeout?

# Of course you can use a timer to check inside the loop
# You can also set a timeout for each fetch

# Terminate after 10 queries

Thread(target=live_write, args=(test_file, 10)).start()

from cmdy import tail

tail_iter = tail(f=True, _=test_file).iter()

for i in range(10):
    print('We got:', tail_iter.next(timeout=1), end='')
    

We got: 0
We got: 1
We got: 2
We got: 3
We got: 4
We got: 5
We got: 6
We got: 7
We got: 8
We got: 9


## Subcommands

In [36]:
from cmdy import git
git.branch(v=True).fg()

* async  92a6209 0.2.2
  master 92a6209 0.2.2


<CmdyResult: ['git', 'branch', '-v']>

In [37]:
# What if I have separate arguments for main and sub-command?

c = git({'git-dir': '.'}, 'branch', {'v': True}).h()
c.cmd

['git', 'branch', 'branch', '--git-dir', '.', '-v']